In [1]:
import numpy as np
import matplotlib.pyplot as plt
import progressbar
import sys

In [2]:
file_path = "Data"
files = [
    "docword.kos.txt",
    "docword.nips.txt",
    "docword.enron.txt",
    ]
max_k = 25
k_iters = 3

In [3]:
def npify(indices:set|list,
          arr_len:int):
    arr = np.zeros(arr_len)
    for index in indices:
        arr[index - 1] = 1
    return arr


def setify(arr):
    threshold = min(0.4, 0.75 * np.max(arr))
    indices = []
    for i in range(len(arr)):
        if arr[i] - threshold >= -1e-3:
            indices.append(i)
    return set(indices)

In [4]:
def jac_dist(v1:set,
             v2:set):    
    union = len(v1.union(v2))
    intersection = len(v1.intersection(v2))
    
    if union == 0:
        return 1
    
    jacc_dist = (union - intersection) / union
    return jacc_dist

In [5]:
def calc_inertia(data,
                 cluster_labels,
                 centroids):
    inertia = 0
    
    for i in range(len(data)):
        inertia += jac_dist(
            data[i],
            centroids[cluster_labels[i]]
            )
    
    return inertia

In [6]:
def custom_kmeans(data:list,
                  k:int,
                  dim:int,
                  max_iter=100,
                  past_centroids:list=None,
                  past_labels:list=None):
    """
    Custom K-Means implementation with a Jaccard Similarity Measure.

    Args:
        data: 
        k: The desired number of clusters.
        max_iter: Maximum number of iterations (default: 100).

    Returns:
        A tuple containing:
            cluster_labels: An array of cluster labels for each data point.
            centroids: The final centroids (cluster centers) after convergence.
            intertia: The inertia for the final centroids and clusters
    """

    if k == 2:
        # Initialize the first centroid randomly
        rnd = np.random.choice(a=len(data),
                                size=1,
                                replace=False)
        centroids = []
        centroids.append(data[rnd[0]])
        
        cluster_labels = [0] * len(data)
    else:
        centroids = past_centroids
        cluster_labels = past_labels
    
    # Add next centroid as the one that is furthest from first centroid
    furthest_pt = data[0]
    max_dist = 0
    for i in range(len(data)):
        pt = data[i]
        c = centroids[cluster_labels[i]]
        
        if jac_dist(c, pt) > max_dist:
            furthest_pt = pt
            max_dist = jac_dist(c, pt)
    
    centroids.append(furthest_pt)
        
    # Re-assign cluster labels
    for i in range(len(data)):
        new_c = 0
        for j in range(len(centroids)):
            if jac_dist(data[i], centroids[new_c]) \
                > jac_dist(data[i], centroids[j]):
                new_c = j
        
        cluster_labels[i] = new_c

    # Try to Converge via kmeans
    for _ in range(max_iter):
        old_centroids = centroids.copy()
        old_labels = cluster_labels
        
        # Calculate Mean of points
        pts_per_cl = [0] * len(centroids)
        sum_per_cl = [np.zeros(dim)] * len(centroids)
        
        for i in range(len(data)):
            c = old_labels[i]
            pts_per_cl[c] += 1
            sum_per_cl[c] += npify(data[i], dim)
        
        new_centroids = []
        for i in range(len(centroids)):
            new_centroids.append(setify(sum_per_cl[i] / pts_per_cl[i]))
        
        # Assign data points to closest centroids
        new_labels = []
        for i in range(len(data)):
            c = 0
            dist = jac_dist(data[i], centroids[c])
            for j in range(len(centroids)):
                if  dist > jac_dist(data[i], centroids[j]):
                    c = j
                    dist = jac_dist(data[i], centroids[c])
            
            new_labels.append(c)
        
        # Stop if new clusters increase inertia
        if calc_inertia(data, cluster_labels=old_labels,
                centroids=old_centroids) < \
            calc_inertia(data, cluster_labels=new_labels,
                centroids=new_centroids):
            cluster_labels = old_labels
            centroids = old_centroids
            break
        else:
            cluster_labels = new_labels
            centroids = new_centroids
        
        # Check for convergence
        if np.all(np.array([jac_dist(c, oc)
                            for c in new_centroids
                            for oc in old_centroids]) < 1e-3):
            break
                
    # Calculate Inertia            
    inertia = calc_inertia(
                data=data,
                cluster_labels=cluster_labels,
                centroids=centroids
                )

    return cluster_labels, centroids, inertia

In [7]:
def kmeans_pipeline(file:str,
                    max_k:int,
                    k_iters:int):
    
    source = open(f'{file_path}/{file}', 'r')
    D = int(next(source).strip())
    W = int(next(source).strip())
    NNZ = int(next(source).strip())
    data = []

    tmp = None
    for line in source:
        d, w, _ = list(map(int, line.strip().split()))
        if d > len(data):
            if tmp is not None:
                data.append(set(tmp))
            tmp = [w]
        else:
            tmp.append(w)
    data.append(set(tmp))

    # Sanity Check
    read_words = sum([len(doc) for doc in data])
    if read_words != NNZ:
        return "Failure: Data Read Improperly"
    
    file_name = file.split('.')[1].upper()
    
    widgets = [f'Clustering on {file_name}: ', progressbar.Percentage(), ' | ',
            progressbar.Timer(), ' | (', progressbar.ETA(), ') ']
    bar = progressbar.ProgressBar(
        maxval=(max_k - 1),
        widgets=widgets)\
            .start()
    
    k_inertia = []
    past_centroids = None
    past_labels = None

    for k in range(2, max_k + 1):
        min_inertia = 10 ** 6
        
        if k == 2:
            for _ in range(k_iters):
                cluster_labels, centroids, inertia = \
                    custom_kmeans(
                        data=data,
                        k=k,
                        dim=W,
                        past_centroids=past_centroids,
                        past_labels=past_labels
                        )

                if min_inertia > inertia:
                    min_inertia = inertia
                    min_centroids = centroids
                    min_labels = cluster_labels
        else:
            cluster_labels, centroids, inertia = \
                custom_kmeans(
                    data=data,
                    k=k,
                    dim=W,
                    past_centroids=past_centroids,
                    past_labels=past_labels
                    )

            min_inertia = inertia
            min_centroids = centroids
            min_labels = cluster_labels
        
        past_centroids = min_centroids
        past_labels = min_labels
        k_inertia.append(min_inertia)
        bar.update(k - 2 + 1)

    print(f'Sparsity of List on {file_name}: {NNZ / (D * W) * 100:.3f} %')
    print(f'Size of List on {file_name}: {sys.getsizeof(data)} bytes')
    print()
    
    return k_inertia            

In [8]:
for file in files:
    k_inertia = kmeans_pipeline(
                    file=file,
                    max_k=max_k,
                    k_iters=k_iters)
    
    file_name = file.split('.')[1].upper()
    
    plt.figure(figsize=(10, 4))
    plt.plot(list(range(2, max_k + 1)),
             k_inertia,
             marker='o', 
             linestyle='-')
    plt.title(f'Clustering on {file_name}')
    plt.xlabel('K')
    plt.ylabel('Inertia (Jaccard Similarity)')
    plt.ylim([max(min(k_inertia) - 0.01 * max(k_inertia), 0), 1.01 * max(k_inertia)])
    plt.savefig(f'Output/KM++ - SR/{file_name}.png')
    plt.close()

Sparsity of List on KOS: 1.491 %
Size of List on KOS: 29336 bytes



Sparsity of List on NIPS: 4.006 %
Size of List on NIPS: 12728 bytes



Sparsity of List on ENRON: 0.331 %
Size of List on ENRON: 351064 bytes

